In [1]:
import deepSI_jax
from deepSI_jax import get_nu_ny_and_auto_norm
from deepSI_jax.utils import find_best_model, RMS_error
import numpy as np

# Data generation
To demonstrate the parallel training option, we generate a simple data-set, similar to the previous examples.

In [2]:
# Generate or load data
np.random.seed(0)  # for reproducibility
U = np.random.randn(10000)  # Input sequence
x = [0, 0]  # Initial state
ylist = []  # Output sequence
for uk in U:
    ek = np.random.normal(loc=0, scale=0.05)
    ylist.append(x[0] + ek)  # Compute output
    x = x[0] / (1.2 + x[1]**2) + x[1] * 0.4, \
        x[1] / (1.2 + x[0]**2) + x[0] * 0.4 + uk  # Advance state
Y = np.array(ylist)  # Output sequence

# Split datasets
Y_train = Y[:9000]
Y_test = Y[9000:]
U_train = U[:9000]
U_test = U[9000:]

# Model initialization
The same principle can be followed for model initialization, with an important difference: the `seed` argument now must be a list with all the initial seeds that will correspond to all the different model initializations.

In [3]:
nu, ny, norm = get_nu_ny_and_auto_norm(Y_train, U_train)
nx = 3
f_dict = dict(hidden_layers=2, nodes_per_layer=32, activation='tanh')
h_dict = dict(feedthrough=False)
seeds = [1, 2, 3]
model = deepSI_jax.SUBNET(nx=nx, ny=ny, nu=nu, norm=norm, f_args=f_dict, h_args=h_dict, use_encoder=False, seed=seeds)

model.set_loss_fun(T=250, T_overlap=50, l2_reg=1e-4)

model.optimization(adam_epochs=200, lbfgs_epochs=200, iprint=-1)  # similarly as before, the iteration numbers can be increased for more accurate results

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


# Parallel training
To initiate parallel training of multiple models, instead of calling the `fit` method of the model, now the `fit_parallel` function needs to be called. here you can also specify the number of training samples that are started at the same time with the `n_jobs` argument that utilizes the maximum available CPU cores by default. Keep in mind that the `fit_parallel` function returns a list of all trained models.

In [4]:
# Train model on data
models = model.fit_parallel(Y_train, U_train, seeds)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


L-BFGS-B done in 172 iterations.
L-BFGS-B done in 172 iterations.
L-BFGS-B done in 183 iterations.


After all models have been trained, we can select the best-performing one based on the training or test errors. We can simply iterate along them and simulate each model on the selected data-set, but there is also a built-in function for that, namely the `deepSI_jax.utils.find_best_model`. This function also simulated the models in parallel for a more time-efficient evaluation process.

In [5]:
# evaluate models on test data to find best
# (this method estimates the initial state values in a highly approximated manner only to compare models)
best_model = find_best_model(models, Y_test, U_test, seeds=seeds)

Evaluating models...



An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.



RTS smoothing, epoch:   1/  1, MSE loss =  0.030045

RTS smoothing, epoch:   1/  1, MSE loss =  0.018657

RTS smoothing, epoch:   1/  1, MSE loss =  0.030966

Final loss MSE (after LBFGS refinement) =  0.017947

Final loss MSE (after LBFGS refinement) =  0.029426
Final loss MSE (after LBFGS refinement) =  0.030356

Errors:
Model 0: RMSE = 0.13089488446712494
Model 1: RMSE = 0.10222359001636505
Model 2: RMSE = 0.13294817507266998
Best model: 1, score: 0.10222359001636505 at seed 2


When the model does not contain an encoder network, the `find_best_model` function roughly estimates the initial states for all models, but after finding the bets-performing structure, a more sophisticated state estimation process is advised, e.g., see below. There are various other options for the  `find_best_model` method that are well documented in the docstring, for example, if the models are compared on the training data-set, the optimized initial states can be used that have been tuned during model training. We will also show an example with a trained encoder network later in this Example.

In [6]:
# evaluate the best model (with a more enhanced initial state estimation method)
x0_test = best_model.learn_x0(U_test, Y_test, RTS_epochs=10, LBFGS_refinement=True, verbosity=False)
Yhat_test, _ = best_model.simulate(x0_test, U_test)
error = RMS_error(Y_test, Yhat_test)

print(f'RMS error: {error:.4f}')

RMS error: 0.1022


# Training encoder networks in parallel
To initialize and train structures with encoders in parallel, the same process can be applied as before.

In [7]:
encoder_dict = dict(hidden_layers=2, nodes_per_layer=16, activation='swish')
n_lag = 20
model = deepSI_jax.SUBNET(nx=nx, ny=ny, nu=nu, norm=norm, f_args=f_dict, h_args=h_dict, use_encoder=True, encoder_args=encoder_dict, encoder_lag=n_lag, seed=seeds)

model.set_loss_fun(T=250, T_overlap=50, l2_reg=1e-4)
model.optimization(adam_epochs=200, lbfgs_epochs=200, iprint=-1)  # only to run fast

# Train model on data
models = model.fit_parallel(Y_train, U_train, seeds)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


L-BFGS-B done in 176 iterations.
L-BFGS-B done in 178 iterations.
L-BFGS-B done in 178 iterations.


Now, we can set the `use_encoder` flag of the `find_best_model` function to `True` in order to utilize the encoders when comparing all models.

In [8]:
# evaluate models on test data to find best
# (now we can use the encoder to estimate the initial states)
best_model = find_best_model(models, Y_test, U_test, seeds=seeds, use_encoder=True)

Evaluating models...



An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Errors:
Model 0: RMSE = 0.11667755246162415
Model 1: RMSE = 0.09886123985052109
Model 2: RMSE = 0.10618513822555542
Best model: 1, score: 0.09886123985052109 at seed 2


And again, the found best model can be evaluated as before.

In [9]:
x0_test = best_model.encoder_estim_x0(Y_test[:n_lag], U_test[:n_lag])
Yhat_test_w_enc, _ = best_model.simulate(x0_test, U_test[n_lag:])  # keep in mind that now we will only simulate from time index n_lag
error = RMS_error(Y_test[n_lag:], Yhat_test_w_enc)
print(f'RMS error: {error:.4f}')

RMS error: 0.0989
